<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/JupyterNotebooks/nuclear_atlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly
running_on_colab = not "JupyterNotebooks" in subprocess.os.getcwd()
if running_on_colab:
    running_on_colab = True
    !wget https://raw.githubusercontent.com/MarcosP7635/Energy/main/ImportedData/nuclear_atlas_full.txt
    parent_dir = "/content/"
else:
    parent_dir = subprocess.os.getcwd().split("JupyterNotebooks")[0]
    sep_char = parent_dir.split("Energy")[-1]
parent_dir

'c:\\Users\\marco\\Documents\\GitHub\\Energy\\'

Now to find where the data starts in the text file. From reading the paper, I know the top of the dataset reads "TABLE I".

In [2]:
file_name = "2023-nuclear-atlas-data-copy-pasted.txt" 
# 2015 version is instead named "nuclear_atlas_full.txt"
if running_on_colab:
    data_dir = parent_dir
else: 
    data_dir = parent_dir + "ImportedData" + sep_char
with open(data_dir + file_name, encoding='utf-8') as file:
    data = file.read().split("\n")
start_line = [x for x in range(len(data)) if "Table 1" == data[x]]
#in 2015 they called in TABLE I and not Table 1, and you use in instead of ==
start_line

[1]

In [3]:
data[start_line[0]:]

['Table 1',
 'Nuclear Isomers with half-life of ≥ 10 ns. @: Half-life for bare or highly-ionized nucleus. † : Energy ordering of the ground state and the isomeric state is unknown.',
 'Z N A X E(keV) J π T 1\ue0092 Eγ(keV) λ Decay mode Reference',
 '4 8 12 Be 2251 (1) 0+ 233(8) ns 142(2) E2 %IT = 100 2018CH38',
 '2251(1) E0 2013JO06',
 '2007SH34',
 '7 9 16 N 120.42 (12) 0− 5.306(28) μs 120.42(12) %IT = 99.99960(4) 2013SO20',
 '%β– = 0.00040(4) 1983MI20',
 '1983GA18',
 '1975PA01',
 '1967BE14',
 '9 9 18 F 1121.36 (15) 5+ 160(4) ns 184 E2 %IT = 100 1979HA60',
 '1972AD01',
 '1972BE37',
 '1967BE14',
 '1967PO09',
 '1959AL99',
 '9 10 19 F 197.143 (4) 5/2+ 88.7(14) ns 87.3 E2 %IT = 100 1968KL05',
 '197.1 1968ST25',
 '1967BE14',
 '1964SU01',
 '1956JO35',
 '1955FI38',
 '10 9 19 Ne 238.27 (11) 5/2+ 18.0(7) ns 238.3 E2 %IT = 100 1969BL02',
 '1967BE14',
 '1957BA09',
 '12 9 21 Mg 205.6 (1) 1/2+ 11.7(5) ns 205.6(1) E2 2019RU02',
 '11 11 22 Na 583.05 (10) 1+ 243(2) ns 583.04(10) E2 %IT = 100 1966SU07'

The rows are formatted as follows:

Z N AX E(keV) J\x02 T1/2 Eγ(keV) λ Decay mode Key No.

The same format is used in both 2015 and 2023. 

Thus, each row begins with an atomic number and number of neutrons, so I can just find the start of a new isomer's data by looking for two consecutive integers at the beginning of a line.

If $J^\pi$ has multiple values, then this will be the end of the line, and the next line will have the half life.

In [4]:
def is_row_start(str):
   arr = str.split(" ")
   if len(arr) > 2:
      return (arr[0].isdigit() and 
              arr[1].isdigit())
   return False
is_row_start(data[1060])

False

In [5]:
data[1060]

'1974CO33'

In [6]:
row_starts = [(r, data[r]) for r in range(len(data)) if is_row_start(data[r])]
row_starts

[(4, '4 8 12 Be 2251 (1) 0+ 233(8) ns 142(2) E2 %IT = 100 2018CH38'),
 (7,
  '7 9 16 N 120.42 (12) 0− 5.306(28) μs 120.42(12) %IT = 99.99960(4) 2013SO20'),
 (12, '9 9 18 F 1121.36 (15) 5+ 160(4) ns 184 E2 %IT = 100 1979HA60'),
 (18, '9 10 19 F 197.143 (4) 5/2+ 88.7(14) ns 87.3 E2 %IT = 100 1968KL05'),
 (24, '10 9 19 Ne 238.27 (11) 5/2+ 18.0(7) ns 238.3 E2 %IT = 100 1969BL02'),
 (27, '12 9 21 Mg 205.6 (1) 1/2+ 11.7(5) ns 205.6(1) E2 2019RU02'),
 (28, '11 11 22 Na 583.05 (10) 1+ 243(2) ns 583.04(10) E2 %IT = 100 1966SU07'),
 (30,
  '11 13 24 Na 472.2074 (8) 1+ 20.18(10) ms 472.2024(8) M3 %IT = 99.95 1980JO11'),
 (34,
  '13 11 24 Al 425.8 (1) 1+ 130.7(13) ms 425.8(1) M3 %IT = 69.6(7) 2011NI18'),
 (40, '9 17 26 F 643.4 (1) (4+) 2.2(1) ms 643.4(1) %IT = 82(11) 2013LE03'),
 (43, '11 15 26 Na 82.40 (4) 1+ 4.35(16) μs 82.40(4) (E2) %IT = 100 2014NIZZ'),
 (46, '13 13 26 Al 228.305 (13) 0+ 6346.02(54) ms %ϵ+%β+ = 100 2020HA30'),
 (54, '15 11 26 P 164.4 (1) 115(9) ns 164.4(1) %IT = 100 2017PE09')

In [7]:
len(row_starts)

2682

They are appear to have around 200 new isomers between the 2015 and 2023 editions of the data table.

For verification, the paper says they list 2469 different isomers, and that they only include isomers with a half life > 10 ns.

Source-

free PDF I put on Zotero:
https://www.zotero.org/groups/4549380/batteries/collections/VUUF9IJG/items/BWM33GUX/attachment/665IXGIJ/reader

DOI: http://dx.doi.org/10.1016/j.nds.2015.08.001)

Assuming there are no gaps in the data, the beginning of one row must be the end of the previous row. 

In [11]:
row = data[row_starts[0][0]]
arr = row.split(" ")
time_units = ['d', 'h', 'min', 'ns', 's', 'y', 'μs', 'ms']
unit_index = [i for i in range(len(arr)) if arr[i] in time_units]
arr

['4',
 '8',
 '12',
 'Be',
 '2251',
 '(1)',
 '0+',
 '233(8)',
 'ns',
 '142(2)',
 'E2',
 '%IT',
 '=',
 '100',
 '2018CH38']

In [15]:
time_units = ['d', 'h', 'min', 'ns', 's', 'y', 'μs', 'ms']
def get_data(row):
   arr = row.split(" ")
   unit_index = [i for i in range(len(arr)) 
                  if arr[i] in time_units]
   if (len(arr) > 4 and arr[0].isdigit() and arr[1].isdigit()
      and not len(unit_index) == 0):
         t_unit = arr[unit_index[0]]
         t_val = arr[unit_index[0]-1].split("(")[0]
         isotope = arr[2] + arr[3]
         E_total = arr[4].split("(")[0]
         return (isotope, E_total, t_val, t_unit)
   return "Improperly Formatted"

data_tuples = [get_data(" ".join(data[row_starts[index][0]:row_starts[index+1][0]]))
                        for index in range(len(row_starts)-1)]
data_tuples

[('12Be', '2251', '233', 'ns'),
 ('16N', '120.42', '5.306', 'μs'),
 ('18F', '1121.36', '160', 'ns'),
 ('19F', '197.143', '88.7', 'ns'),
 ('19Ne', '238.27', '18.0', 'ns'),
 ('21Mg', '205.6', '11.7', 'ns'),
 ('22Na', '583.05', '243', 'ns'),
 ('24Na', '472.2074', '20.18', 'ms'),
 ('24Al', '425.8', '130.7', 'ms'),
 ('26F', '643.4', '2.2', 'ms'),
 ('26Na', '82.40', '4.35', 'μs'),
 ('26Al', '228.305', '6346.02', 'ms'),
 ('26P', '164.4', '115', 'ns'),
 ('31Mg', '49.93', '12.0', 'ns'),
 ('31Mg', '461.0', '10.5', 'ns'),
 ('32Mg', '1057', '7-26', 'ns'),
 ('32Al', '956.6', '200', 'ns'),
 ('32Si', '5581', '27', 'ns'),
 ('33Si', '1434.9', '10.2', 'ns'),
 ('34Al', '46.47', '22.1', 'ms'),
 ('34Si', '2718.52', '19.4', 'ns'),
 ('34Cl', '146.36', '31.99', 'min'),
 ('37K', '1380.25', '10.4', 'ns'),
 ('38Cl', '671.365', '715', 'ms'),
 'Improperly Formatted',
 ('38K', '3458.10', '21.95', 'μs'),
 ('40K', '1643.638', '0.336', 'μs'),
 ('42Sc', '616.782', '61.7', 's'),
 ('43S', '320.7', '415', 'ns'),
 ('43K', 

In [10]:
set([x[3] for x in data_tuples if not type(x) == str])

{'d', 'h', 'min', 'ms', 'ns', 's', 'y', 'μs'}

In [16]:
bad_rows = [r for r in range(len(data_tuples)) if type(data_tuples[r]) == str]
bad_rows

[24,
 37,
 67,
 112,
 149,
 234,
 257,
 293,
 305,
 306,
 317,
 342,
 423,
 430,
 449,
 463,
 481,
 492,
 507,
 556,
 569,
 710,
 711,
 877,
 908,
 954,
 974,
 1021,
 1302,
 1350,
 1380,
 1404,
 1432,
 1601,
 1760,
 2100,
 2247,
 2279,
 2359,
 2459,
 2479,
 2495,
 2581,
 2609,
 2628,
 2630,
 2631,
 2632,
 2638,
 2652,
 2653,
 2661,
 2662]

In [17]:
[data[row_starts[bad_row][0]] for bad_row in bad_rows]

['19 19 38 K 130.22 (16) 0+ 924.33(27) ms* 130.1(2) %ϵ+%β+ = 99.9670(43) 2020HA30',
 '21 23 44 Sc 271.240 (10) 6+ 58.61(10) h* 271.24(1) %IT = 98.80(7) 1969RA16',
 '26 31 57 Fe 14.4129 (6) 3/2− 98.04(20) ns* 14.4129(6) M1+E2 %IT = 100 2021DU16',
 '30 37 67 Zn 93.312 (5) 1/2− 9.142(24) μs* 93.311(5) E2 %IT = 100 2015CH57',
 '30 41 71 Zn 157.7 (13) 9/2+ 4.140(15) h* 386 %β−= 100 2017KR01',
 '32 49 81 Ge 679.14 (4) (1/2+) 7.6(6) s* %β−= 99.5(5) 1981HO24',
 '37 46 83 Rb 42.0780 (20) 9/2+ >7.8 ms* 42.078(2) M2 %IT = 100 1967MOZZ',
 '38 49 87 Sr 388.5287 (23) 1/2− 2.805(1) h* 388.5276(23) M4 %IT = 99.70(8) 2021KR05',
 '43 45 88 Tc 70.4 (31)* (6+) 6.4(8) s* %ϵ+%β+ = 100 2019VI05*',
 '43 45 88 Tc 95* (4+) 146(12) ns* 95 %IT = 100 2019VI05*',
 '39 51 90 Y 681.67 (10) 7+ 3.196(12) h* 479.532(10) M4(+E5) %IT = 99.9982(2) 2020KR06',
 '45 46 91 Rh 172.9 (4) (1/2−) 1.60(3) s* %IT = ? 2019PA16',
 '38 59 97 Sr 830.80 (22) (9/2+) 513(8) ns* 308.3(2) (E2) %IT = 100 2019ES04',
 '47 50 97 Ag # 618 (38) (1

As noted in the paper, the asterisk can indicate that there are multiple values that disagree and/or that the existing measurements are very low precision. 

Jain et al. said "the statistics are very low" for 101Y, so I will assume this means few measurements were taken and this isotope will be omitted. There are similar footnotes for 102Sn and 121In, so these two will also be omitted. 

Now to further filter the data.

In [18]:
def measured(input):
    try: 
        float(input)
        return True
    except:
        return False
measured("1000"), measured("cat")

(True, False)

These are the indices of the isotopes that have measurements for both their Decay energy and half life.

In [20]:
content = data_tuples[0]
measured(content[1])

True

In [21]:
good_tuple_indices = [index  for index, content in enumerate(data_tuples)
                        if (measured(content[2].split("(")[0]) and 
                        measured(content[1]))]
len(good_tuple_indices)

2140

In [22]:
unit_dict = {'d':24*3600, 'h':3600, 'min':60, 'ms':.001, 'ns':10**-9, 's':1, 'y':365*24*3600, 'μs':10**-6}
isotopes = [content[0] for index, content in enumerate(data_tuples) 
            if index in good_tuple_indices]
total_energies_keV = [float(content[1]) for index, content in enumerate(data_tuples) 
            if index in good_tuple_indices]
half_lives_sec = [float(content[2].split("(")[0]) * unit_dict[content[3]] 
                for index, content in enumerate(data_tuples) 
                if index in good_tuple_indices]
data_dict = {"Isotope":isotopes, "Decay Energy (keV)":total_energies_keV, "Half-life (seconds)":half_lives_sec}
df = pd.DataFrame(data_dict)
df

,Isotope,Decay Energy (keV),Half-life (seconds)
0,12Be,2251.000,2.330000e-07
1,16N,120.420,5.306000e-06
2,18F,1121.360,1.600000e-07
3,19F,197.143,8.870000e-08
4,19Ne,238.270,1.800000e-08
...,...,...,...
2135,228Ac,20.600,1.150000e-07
2136,228Ac,356.000,1.190000e+02
2137,234Ac,140.000,9.300000e+01
2138,228Ac,620.000,1.490000e+02


In [26]:
df.shape

(2140, 3)

In [27]:
[x for x in range(df.shape[0]) if df["Isotope"][x] == "178Hf"]

[1375, 1376, 1377, 1378]

In [38]:
df.to_csv(parent_dir + "ExportedData" + sep_char + "2023_atlas_of_nuclear_isomers.csv")

In [39]:
fig = px.scatter(df, x = "Half-life (seconds)", y = "Decay Energy (keV)", 
    hover_name = "Isotope", log_x = True, log_y = True, 
    title=('Interactive Plot of Nuclear Isomers<br>' +
    '<sup>Data Retrieved from Jain et al. 2015. ' + 
    '<a href="https://doi.org/10.1016/j.nds.2015.08.001">' + 
    'DOI:10.1016/j.nds.2015.08.001</a></sup>'))
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>Hal' ... 'ergy (keV)=%{y}<extra></extra>'),
              'hovertext': array(['12Be', '16N', '18F', ..., '234Ac', '228Ac', '255Db'], dtype=object),
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([2.330e-07, 5.306e-06, 1.600e-07, ..., 9.300e+01, 1.490e+02, 2.800e-03]),
              'xaxis': 'x',
              'y': array([2251.  ,  120.42, 1121.36, ...,  140.  ,  620.  ,    2.8 ]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': ('Interactive Plot of Nuclear Is' ... '16/j.nds.2015.08.001</a></sup>')},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Half-life (seconds)'}, 'type': 'log'},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Decay Energy (keV)'}, 'type': 'log'}}
})

Export plot as an interactive html file

In [40]:
plot_dir = parent_dir + sep_char + "ExportedData" + sep_char + "Plots" + sep_char
plot_path = plot_dir + "2023_Nuclear_Isomers.html"
fig.write_html(plot_path)

Check if this is running on my local machine. If so, upload it to the internet.

        1 file(s) copied.


In [44]:
if "marco" in parent_dir:
    !cd .. && cd .. && copy Energy\ExportedData\Plots\2023_Nuclear_Isomers.html plots\2023_Nuclear_Isomers.html
    print("Copied to new directory")
    !cd .. && cd .. && cd plots && git add * && git commit -m "programmatic update" && git push
    print("Uploaded to internet")

        1 file(s) copied.
Copied to new directory


[main 35f0020] programmatic updateUploaded to internet


To https://github.com/MarcosP7635/plots.git
   8adb30b..35f0020  main -> main



 1 file changed, 14 insertions(+)
 create mode 100644 2023_Nuclear_Isomers.html


This plot can be viewed at https://marcosp7635.github.io/plots/Nuclear_Isomers.html and https://mp7635plots.web.app/Nuclear_Isomers.html

Now to filter the isomers to show only those with relevant half-lives

In [17]:
long_lived_rows = [row for row in range(df.shape[0]) 
                    if df.loc[row, "Half-life (seconds)"] > unit_dict["y"]]
long_lived_isotopes = [df.loc[row, "Isotope"] for row in long_lived_rows]
long_lived_isotopes

['93Nb',
 '102Rh',
 '108Ag',
 '113Cd',
 '121Sn',
 '178Hf',
 '180Ta',
 '186Re',
 '192Ir',
 '210Bi',
 '242Am']

In [18]:
def long_lived(isotope):
    return isotope in long_lived_isotopes

In [19]:
filtered_df = pd.DataFrame()
for col in df.columns:
    filtered_df[col] = [df.loc[row, col] for row in range(df.shape[0]) 
            if df.loc[row, "Isotope"] in long_lived_isotopes]
filtered_df

,Isotope,Decay Energy (keV),Half-life (seconds)
0,93Nb,30.7700,5.083603e+08
1,102Rh,41.9400,1.890000e-08
2,102Rh,140.7300,1.180077e+08
3,108Ag,109.4660,1.381277e+10
4,108Ag,215.3820,4.580000e-08
5,113Cd,263.5400,4.380350e+08
6,113Cd,316.2060,1.080000e-08
7,121Sn,6.3100,1.384430e+09
8,121Sn,1998.3000,5.300000e-06
9,121Sn,2221.1000,5.000000e-07


In [20]:
def rewrite_isotope(isotope):
    mass = "".join([x for x in isotope if x.isdigit()])
    name = isotope.split(mass)[1]
    return name + "-" + mass
filtered_df["Isotope"] = [rewrite_isotope(isotope) 
                        for isotope in filtered_df["Isotope"]]
filtered_df

,Isotope,Decay Energy (keV),Half-life (seconds)
0,Nb-93,30.7700,5.083603e+08
1,Rh-102,41.9400,1.890000e-08
2,Rh-102,140.7300,1.180077e+08
3,Ag-108,109.4660,1.381277e+10
4,Ag-108,215.3820,4.580000e-08
5,Cd-113,263.5400,4.380350e+08
6,Cd-113,316.2060,1.080000e-08
7,Sn-121,6.3100,1.384430e+09
8,Sn-121,1998.3000,5.300000e-06
9,Sn-121,2221.1000,5.000000e-07


In [21]:
title=('Long-lived Nuclear Isomers<br><sup>Data Retrieved from ' +
     'Jain et al. 2015. <a href="https://doi.org/10.1016/j.nds.2015.08.001">' + 
    'DOI:10.1016/j.nds.2015.08.001</a></sup>')
fig = px.scatter(filtered_df, log_y = True, y="Half-life (seconds)", x="Isotope",
            color = "Decay Energy (keV)", title=title)
fig

The below graph is more useful to me. It is easier to spot isotopes that have isomers close in decay energy but very large difference in half life, which is desirable for energy storage. In the plot, for each isotopes this would show up as points that are vertically close together but have different colors. 
Ideally, I could make such a plot for every isotope and simply scroll to the left and right to view prospects. Alternatively, I could do this programmatically by taking the ratio between change in half life and change in decay energy.

In [45]:
color_axis = "Half-life in log<sub>10</sub>(year)"
filtered_df[color_axis] = np.log10(filtered_df["Half-life (seconds)"] / (24 * 365 * 3600))
title=('Long-lived Nuclear Isomers<br><sup>Data Retrieved from ' +
     'Jain et al. 2015. <a href="https://doi.org/10.1016/j.nds.2015.08.001">' + 
    'DOI:10.1016/j.nds.2015.08.001</a></sup>')
fig = px.scatter(filtered_df, log_y = True, y="Decay Energy (keV)", x="Isotope",
            color = color_axis, title=title)
fig

Thus Hf-178, Ta-180, Ir-192, and Cd-113 seem especially promising.

Similar to before, export as an interactive HTML file and optionally upload it to the internet.

In [42]:
plot_dir = parent_dir + sep_char + "ExportedData" + sep_char + "Plots" + sep_char
plot_path = plot_dir + "Long_Lived_Isomers.html"
fig.write_html(plot_path)

In [43]:
if "engin" in parent_dir:
    !cd .. && cd .. && cp Energy/ExportedData/Plots/Long_Lived_Isomers.html plots 
    print("Copied to new directory")
    !cd .. && cd .. && cd plots && git add * && git commit -m "programmatic update" && git push
    print("Uploaded to internet")

Copied to new directory
[main 62838c6] programmatic update
 1 file changed, 1 insertion(+), 1 deletion(-)
Uploaded to internet


To https://github.com/MarcosP7635/plots.git
   7fb8d0f..62838c6  main -> main


This plot can be viewed at https://marcosp7635.github.io/plots/Long_Lived_Isomers.html and https://mp7635plots.web.app/Long_Lived_Isomers.html

To quantify the metric for evaluating each pair of energy levels, we will consider the change in half-life and the change in decay energy. The energy level from which energy will be released will have a half-life of most $t_{min}$ and the energy level that will be used to store energy will have a half-life of at least $t_{max}$ (stable energy levels effectively have an infinite half-life).

When storing energy, the isotope will have an energy $E_c$ and just before energy is released, it will have an energy $E_r$. Thus the system will need to supply an energy of at least $E_r - E_c$ to release an energy $E_r$. Assuming that the battery is charged, this would result in the energy available to the system increasing by a factor of 
\begin{equation}
\text{Energy multiplication factor} < \frac{E_r}{E_r-E_c}
\end{equation}
Assuming there is only 1 step in the decay chain of any of the isomers, power released will be at most
\begin{equation}
P \le P_{max} = N\frac{E_r\ln 2}{t}N_A
\end{equation}
where $t$ is the half-life of the energy level with a decay energy $E$ of which there are $N$ moles, and $N_A$ is Avogadro's constant.
So instead of constraining the half-life of the higher energy level, I can simply calculate the ideal power produced, $P_{max}$. Of course, this assumes there are no side reactions as a result of the energy release.
This also allows me to instead find the ideal power multiplication factor, $p_{m}$
\begin{equation}
p_m =  N\frac{E_r\ln 2}{(E_r-E_c)t}N_A
\end{equation}
If the daughter isotopes are not stable, then the entire decay chain should be taken into account. Luckily, I have already written a streamlit application that shows such results: 
https://marcosp7635-power-densities-plot-power-de1e9n.streamlit.app/ which shows the results from the analysis done in the Jupyter notebook named make_comprehensive_power.ipynb, which is in the same folder as this notebook.

In [55]:
[x for x in set(df["Isotope"]) if "*" in x]

['181Hf*', '106Cd*']

In [ ]:
[e for e in isotopes if "121Sn" in e]

['121Sn', '121Sn', '121Sn', '121Sn']